在上一课中，将数据管道纳入了交易算法。现在是时候定义算法如何将使用流水线生成的数据来重新平衡投资组合。我们的目标是根据尾部12个月股利与股价比率找到最大化回报的目标投资组合，同时保持由一组规则或约束条件定义的特定结构。这通常被称为投资组合优化问题。

`Optimize API`使我们能够轻松地将管道输出转化为目标和一组约束条件。 然后，我们可以使用`order_optimal_portfolio`将当前的投资组合转换为符合规范的目标投资组合。

第一步是定义一个目标。我们将使用`MaximizeAlpha`，它将尝试将资金分配给与尾部12个月股利与股价比率成比例的资产。

接下来，我们需要指定我们希望目标投资组合满足的约束列表。 我们首先在初始化时定义一些阈值并将它们存储在我们的上下文变量中：

现在，我们可以使用我们上面定义的阈值来指定`rebalance`中的约束条件：

最后，我们可以将我们的目标和约束列表传递给`order_optimal_portfolio`以计算目标投资组合，并发布将我们当前投资组合转换为最佳状态所需的订单：

# 风险管理

除了对目标投资组合的结构设置限制外，我们还希望限制可能影响其业绩的共同风险因子敞口。例如，由于stocktwits的情绪数据具有短暂性，我们打算利用情绪分数高峰期，算法可能会面临短期逆转风险。

我们将使用Quantopian的风险模型来管理我们投资组合的共同风险因子。风险模型计算资产的16种不同风险因子敞口：11个部门因子和5个主题因子（包括短期逆转）。我们可以使用`risk_loading_pipeline`函数在算法中使用这些数据，该函数返回一个数据管道，该数据管道为风险模型中的每个因子生成一列输出。

与数据管道类似，需要将风险数据管道附加到我们的算法，并提供一个名称来识别它。 然后，在`before_trading_start`中获取它的输出并将其存储在`context`中：

下一步是将`RiskModelExposure`约束添加到我们的投资组合优化逻辑。 该约束采用风险模型生成的数据，并对模型中包含的每个因子设置目标投资组合的整体风险敞口限制。

最后，以下算法包含策略和投资组合构建逻辑，并准备好进行回测。

本部分主要测试`risk_loading_pipeline`和`RiskModelExposure`优化问题求解，相对于原案例有较大改动。

In [1]:
%load_ext zipline

In [2]:
%%zipline --start 2018-1-8 --end 2018-6-8

# Import Algorithm API functions
from zipline.api import (
    attach_pipeline,
    pipeline_output,
    order_optimal_portfolio,
    schedule_function,
    date_rules,
    time_rules,
)

# Import Optimize API module
import zipline.optimize as opt

# Pipeline imports
from zipline.pipeline import Pipeline
#from zipline.pipeline.data.psychsignal import stocktwits
from zipline.pipeline.builtin import QTradableStocksUS, trailing_dividend_yield
from zipline.pipeline.factors import SimpleMovingAverage

# 运行时函数在zipline.pipeline.builtin模块
#from zipline.pipeline.filters import QTradableStocksUS
from zipline.pipeline.risk import risk_loading_pipeline

import numpy as np
import pandas as pd


def make_pipeline():

    base_universe = QTradableStocksUS()

    # 尾部12个月每股股利平均值/股价
    tdy = trailing_dividend_yield()
    # 只是利用了Index作为股票总体
    return Pipeline(
        columns={
            'tdy': tdy,
        },
        # Set screen as the intersection between our filter
        # and trading universe
        screen=(
            base_universe
            & tdy.notnull()  # 选择非空的那一部分与可交易总体的交集
        ))

def initialize(context):
    # Constraint parameters
    context.max_leverage = 1.5 # 尽管限制总杠杆为1.5，由于算法尚未考虑cash因素，所以部分日期会有一定程度的偏离
    context.max_pos_size = 0.015
    context.max_turnover = 0.95

    # Attach data pipelines
    attach_pipeline(
        make_pipeline(),
        'data_pipe'
    )
    attach_pipeline(
        risk_loading_pipeline(),
        'risk_pipe'
    )

    # Schedule rebalance function
    schedule_function(
        rebalance,
        date_rules.week_start(),
        time_rules.market_open(),
    )

def handle_data(context, data):
    t = context.get_datetime('Asia/Shanghai')
    print(t, '股票数量与绝对值权重合计')
    s = context.get_current_allocations()
    print(s.abs().sum())
    print(s)

def before_trading_start(context, data):
    # Get pipeline outputs and
    # store them in context
    output = pipeline_output('data_pipe')
    
    context.alphas = pd.Series(np.random.random_integers(-100,100, len(output)) / 100,
                               index = output.index)
    context.risk_factor_betas = pipeline_output('risk_pipe').fillna(0.0)



def rebalance(context, data):
    # Create MaximizeAlpha objective using
    # tdy data from pipeline output
    objective = opt.MaximizeAlpha(
      context.alphas
    )

    # Create position size constraint
    constrain_pos_size = opt.PositionConcentration.with_equal_bounds(
        -context.max_pos_size,
        context.max_pos_size
    )
    
    # Ensure long and short books
    # are roughly the same size
    #dollar_neutral = opt.DollarNeutral()

    # Constrain target portfolio's leverage
    max_leverage = opt.MaxGrossExposure(context.max_leverage)

    # Constrain portfolio turnover
    # 没有实现周转率限制
    # max_turnover = opt.MaxTurnover(context.max_turnover)

    # Constrain target portfolio's risk exposure
    # By default, max sector exposure is set at
    # 0.2, and max style exposure is set at 0.4
    factor_risk_constraints = opt.RiskModelExposure(
        context.risk_factor_betas,
        #version=opt.Newest # 弃用版本参数
    )

    # Rebalance portfolio using objective
    # and list of constraints
    order_optimal_portfolio(
        objective=objective,
        constraints=[
            max_leverage,
            #dollar_neutral,
            #constrain_pos_size,
            factor_risk_constraints,
        ]
    )


2018-05-08 15:00:00+08:00 股票数量与绝对值权重合计
0.0
Series([], dtype: float64)
2018-05-09 15:00:00+08:00 股票数量与绝对值权重合计
0.0
Series([], dtype: float64)
2018-05-10 15:00:00+08:00 股票数量与绝对值权重合计
0.0
Series([], dtype: float64)
2018-05-11 15:00:00+08:00 股票数量与绝对值权重合计
0.0
Series([], dtype: float64)
2018-05-14 15:00:00+08:00 股票数量与绝对值权重合计
0.0
Series([], dtype: float64)
2018-05-15 15:00:00+08:00 股票数量与绝对值权重合计
1.5144219564404884
华侨城Ａ(000069)     0.088009
宜华健康(000150)     0.021097
中鼎股份(000887)    -0.062748
大庆华科(000985)     0.096673
世荣兆业(002016)     0.093121
登海种业(002041)    -0.038387
国脉科技(002093)    -0.077009
北化股份(002246)    -0.028382
完美世界(002624)     0.034476
福安药业(300194)     0.031808
宜通世纪(300310)     0.117336
中潜股份(300526)    -0.013402
贝达药业(300558)     0.093507
国金证券(600109)    -0.051034
江苏阳光(600220)     0.045268
*ST正源(600321)   -0.039962
澳柯玛(600336)     -0.149395
陆家嘴(600663)      0.032284
上海凤凰(600679)     0.062899
鲁北化工(600727)    -0.063395
华域汽车(600741)    -0.085913
平煤股份(601666)    -0.021733
光大证券(601788)     0.0

2018-06-05 15:00:00+08:00 股票数量与绝对值权重合计
1.5379174910947064
中国宝安(000009)    0.049893
东旭蓝天(000040)    0.161058
华天酒店(000428)   -0.019668
京东方Ａ(000725)   -0.042736
锌业股份(000751)    0.037462
航锦科技(000818)   -0.063495
赣能股份(000899)   -0.037224
史丹利(002588)    -0.088096
神州泰岳(300002)    0.062797
中航电测(300114)   -0.068126
汇川技术(300124)    0.053768
秀强股份(300160)   -0.064966
佳云科技(300242)    0.033927
美尚生态(300495)   -0.061805
新美星(300509)    -0.046219
黄山旅游(600054)   -0.082592
鄂尔多斯(600295)   -0.052130
浙大网新(600797)    0.079803
力帆股份(601777)    0.051795
晶方科技(603005)    0.223923
康尼机电(603111)   -0.062849
普莱柯(603566)     0.055260
晨光文具(603899)   -0.038325
dtype: float64
2018-06-06 15:00:00+08:00 股票数量与绝对值权重合计
1.5078949466395344
中国宝安(000009)    0.049922
东旭蓝天(000040)    0.158664
京东方Ａ(000725)   -0.042177
锌业股份(000751)    0.038085
航锦科技(000818)   -0.063422
赣能股份(000899)   -0.037181
史丹利(002588)    -0.087701
神州泰岳(300002)    0.062620
中航电测(300114)   -0.066753
汇川技术(300124)    0.053131
秀强股份(300160)   -0.064019
佳云科技(300242)    0.0

,algo_volatility,algorithm_period_return,alpha,benchmark_period_return,benchmark_volatility,beta,capital_used,ending_cash,ending_exposure,ending_value,...,short_exposure,short_value,shorts_count,sortino,starting_cash,starting_exposure,starting_value,trading_days,transactions,treasury_period_return
2018-05-08 07:00:00+00:00,NaN,0.000000,NaN,0.011603,NaN,NaN,0.000000e+00,1.000000e+07,0.000,0.000,...,0.00,0.00,0,NaN,1.000000e+07,0.000,0.000,1,[],0.0
2018-05-09 07:00:00+00:00,0.000000,0.000000,0.000000,0.009761,0.150684,0.000000,0.000000e+00,1.000000e+07,0.000,0.000,...,0.00,0.00,0,NaN,1.000000e+07,0.000,0.000,2,[],0.0
2018-05-10 07:00:00+00:00,0.000000,0.000000,0.000000,0.015353,0.106715,0.000000,0.000000e+00,1.000000e+07,0.000,0.000,...,0.00,0.00,0,NaN,1.000000e+07,0.000,0.000,3,[],0.0
2018-05-11 07:00:00+00:00,0.000000,0.000000,0.000000,0.010080,0.119479,0.000000,0.000000e+00,1.000000e+07,0.000,0.000,...,0.00,0.00,0,NaN,1.000000e+07,0.000,0.000,4,[],0.0
2018-05-14 07:00:00+00:00,0.000000,0.000000,0.000000,0.019588,0.114424,0.000000,0.000000e+00,1.000000e+07,0.000,0.000,...,0.00,0.00,0,NaN,1.000000e+07,0.000,0.000,5,[],0.0
2018-05-15 07:00:00+00:00,0.006000,-0.000926,-0.039322,0.023449,0.102347,0.000449,-1.077651e+04,9.989223e+06,1518.990,1518.990,...,-7564340.39,-7564340.39,13,-6.480741,1.000000e+07,0.000,0.000,6,"[{'amount': 108687, 'dt': 2018-05-15 07:00:00+...",0.0
2018-05-16 07:00:00+00:00,0.047610,0.006795,0.379646,0.015299,0.117413,-0.243668,0.000000e+00,9.989223e+06,78727.550,78727.550,...,-7520568.85,-7520568.85,13,44.086904,9.989223e+06,1518.990,1518.990,7,[],0.0
2018-05-17 07:00:00+00:00,0.044220,0.007138,0.270843,0.007788,0.121303,-0.182715,0.000000e+00,9.989223e+06,82154.993,82154.993,...,-7511628.54,-7511628.54,13,43.303428,9.989223e+06,78727.550,78727.550,8,[],0.0
2018-05-18 07:00:00+00:00,0.042138,0.006507,0.273646,0.017963,0.123264,-0.180306,0.000000e+00,9.989223e+06,75847.283,75847.283,...,-7584677.08,-7584677.08,13,30.850195,9.989223e+06,82154.993,82154.993,9,[],0.0
2018-05-21 07:00:00+00:00,0.065465,-0.003197,0.052118,0.022704,0.116973,-0.228563,-2.092636e+03,9.987131e+06,-19097.030,-19097.030,...,-7686148.16,-7686148.16,13,-1.616112,9.987131e+06,75847.283,75847.283,10,[],0.0
